<a href="https://colab.research.google.com/github/fabiopernisi/HypoxiaClassification/blob/main/Pensioni_Transcript3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.stem.snowball import ItalianStemmer
import spacy
import copy
import numpy as np
import torch
from google.colab import files
import pickle
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
pensioni_df = pd.read_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/pensioni.xlsx")

audio_df = pd.read_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/temp_audio.xlsx")
open_df = pd.read_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/temp_open.xlsx")
choseaudio_df = pd.read_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/temp_choseaudio.xlsx")
choseopen_df = pd.read_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/temp_choseopen.xlsx")

#Make copies
temp_audio_df = audio_df.copy(deep=True)
temp_open_df = open_df.copy(deep=True)
temp_choseaudio_df = choseaudio_df.copy(deep=True)
temp_choseopen_df = choseopen_df.copy(deep=True)

In [ ]:
pensioni_df.columns

Index(['IDU', 'STARTTIME', 'ENDTIME', 'TOTALDURATION', 'p_sesso', 'p_r_eta',
       'p_r_eta_6', 'generation', 'random', 'lega', 'p_comune_istat',
       'p_geo_cap', 'p_r_ampiezza6', 'r_capoluogo', 'a_istat_reg',
       'a_istat_prov', 'zona_5istat', 'p_occ_principale', 'p_prof',
       'p_autocol', 'p_voto_pol22', 'p_int_voto', 'p_scolarita', 'scol2', 'd0',
       'introaudio', 'data0', 'ora0', 'd1ap', 'd1', 'd1aperta', 'd1recorder',
       'd1rec', 'data1', 'ora1', 'd2ap', 'd2', 'd2aperta', 'd2recorder',
       'd2rec', 'data2', 'ora2', 'd3ap', 'd3', 'd3aperta', 'd3recorder',
       'd3rec', 'data3', 'ora3', 'etapensione', 'annicontributi', 'platform'],
      dtype='object')

In [ ]:
temp_pensioni = pensioni_df.copy(deep=True)

In [ ]:
d1ap_loc = temp_pensioni.columns.get_loc('d1ap')
temp_pensioni.insert(d1ap_loc+1, "label_d1_emotion", np.zeros(temp_pensioni.shape[0]))

label_d1ap_emotion_loc = temp_pensioni.columns.get_loc('label_d1_emotion')
temp_pensioni.insert(label_d1ap_emotion_loc+1, "score_d1_anger", np.zeros(temp_pensioni.shape[0]))

score_d1ap_anger_loc = temp_pensioni.columns.get_loc('score_d1_anger')
temp_pensioni.insert(score_d1ap_anger_loc+1, "score_d1_fear", np.zeros(temp_pensioni.shape[0]))

score_d1ap_fear_loc = temp_pensioni.columns.get_loc('score_d1_fear')
temp_pensioni.insert(score_d1ap_fear_loc+1, "score_d1_sadness", np.zeros(temp_pensioni.shape[0]))

score_d1ap_sadness_loc = temp_pensioni.columns.get_loc('score_d1_sadness')
temp_pensioni.insert(score_d1ap_sadness_loc+1, "score_d1_joy", np.zeros(temp_pensioni.shape[0]))

score_d1ap_joy_loc = temp_pensioni.columns.get_loc('score_d1_joy')
temp_pensioni.insert(score_d1ap_joy_loc+1, "score_d1_positive_sent", np.zeros(temp_pensioni.shape[0]))

In [ ]:
# d1aperta_loc = temp_pensioni.columns.get_loc('d1aperta')
# temp_pensioni.insert(d1aperta_loc+1, "label_d1aperta_emotion", np.zeros(temp_pensioni.shape[0]))

# label_d1aperta_emotion_loc = temp_pensioni.columns.get_loc('label_d1aperta_emotion')
# temp_pensioni.insert(label_d1aperta_emotion_loc+1, "score_d1aperta_anger", np.zeros(temp_pensioni.shape[0]))

# score_d1aperta_anger_loc = temp_pensioni.columns.get_loc('score_d1aperta_anger')
# temp_pensioni.insert(score_d1aperta_anger_loc+1, "score_d1aperta_fear", np.zeros(temp_pensioni.shape[0]))

# score_d1aperta_fear_loc = temp_pensioni.columns.get_loc('score_d1aperta_fear')
# temp_pensioni.insert(score_d1aperta_fear_loc+1, "score_d1aperta_sadness", np.zeros(temp_pensioni.shape[0]))

# score_d1aperta_sadness_loc = temp_pensioni.columns.get_loc('score_d1aperta_sadness')
# temp_pensioni.insert(score_d1aperta_sadness_loc+1, "score_d1aperta_joy", np.zeros(temp_pensioni.shape[0]))

# score_d1aperta_joy_loc = temp_pensioni.columns.get_loc('score_d1aperta_joy')
# temp_pensioni.insert(score_d1aperta_joy_loc+1, "score_d1aperta_positive_sent", np.zeros(temp_pensioni.shape[0]))

In [ ]:
d2ap_loc = temp_pensioni.columns.get_loc('d2ap')
temp_pensioni.insert(d2ap_loc+1, "label_d2_emotion", np.zeros(temp_pensioni.shape[0]))

label_d2ap_emotion_loc = temp_pensioni.columns.get_loc('label_d2_emotion')
temp_pensioni.insert(label_d2ap_emotion_loc+1, "score_d2_anger", np.zeros(temp_pensioni.shape[0]))

score_d2ap_anger_loc = temp_pensioni.columns.get_loc('score_d2_anger')
temp_pensioni.insert(score_d2ap_anger_loc+1, "score_d2_fear", np.zeros(temp_pensioni.shape[0]))

score_d2ap_fear_loc = temp_pensioni.columns.get_loc('score_d2_fear')
temp_pensioni.insert(score_d2ap_fear_loc+1, "score_d2_sadness", np.zeros(temp_pensioni.shape[0]))

score_d2ap_sadness_loc = temp_pensioni.columns.get_loc('score_d2_sadness')
temp_pensioni.insert(score_d2ap_sadness_loc+1, "score_d2_joy", np.zeros(temp_pensioni.shape[0]))

score_d2ap_joy_loc = temp_pensioni.columns.get_loc('score_d2_joy')
temp_pensioni.insert(score_d2ap_joy_loc+1, "score_d2_positive_sent", np.zeros(temp_pensioni.shape[0]))

In [ ]:
# d2aperta_loc = temp_pensioni.columns.get_loc('d2aperta')
# temp_pensioni.insert(d2aperta_loc+1, "label_d2aperta_emotion", np.zeros(temp_pensioni.shape[0]))

# label_d2aperta_emotion_loc = temp_pensioni.columns.get_loc('label_d2aperta_emotion')
# temp_pensioni.insert(label_d2aperta_emotion_loc+1, "score_d2aperta_anger", np.zeros(temp_pensioni.shape[0]))

# score_d2aperta_anger_loc = temp_pensioni.columns.get_loc('score_d2aperta_anger')
# temp_pensioni.insert(score_d2aperta_anger_loc+1, "score_d2aperta_fear", np.zeros(temp_pensioni.shape[0]))

# score_d2aperta_fear_loc = temp_pensioni.columns.get_loc('score_d2aperta_fear')
# temp_pensioni.insert(score_d2aperta_fear_loc+1, "score_d2aperta_sadness", np.zeros(temp_pensioni.shape[0]))

# score_d2aperta_sadness_loc = temp_pensioni.columns.get_loc('score_d2aperta_sadness')
# temp_pensioni.insert(score_d2aperta_sadness_loc+1, "score_d2aperta_joy", np.zeros(temp_pensioni.shape[0]))

# score_d2aperta_joy_loc = temp_pensioni.columns.get_loc('score_d2aperta_joy')
# temp_pensioni.insert(score_d2aperta_joy_loc+1, "score_d2aperta_positive_sent", np.zeros(temp_pensioni.shape[0]))

In [ ]:
d3ap_loc = temp_pensioni.columns.get_loc('d3ap')
temp_pensioni.insert(d3ap_loc+1, "label_d3_emotion", np.zeros(temp_pensioni.shape[0]))

label_d3ap_emotion_loc = temp_pensioni.columns.get_loc('label_d3_emotion')
temp_pensioni.insert(label_d3ap_emotion_loc+1, "score_d3_anger", np.zeros(temp_pensioni.shape[0]))

score_d3ap_anger_loc = temp_pensioni.columns.get_loc('score_d3_anger')
temp_pensioni.insert(score_d3ap_anger_loc+1, "score_d3_fear", np.zeros(temp_pensioni.shape[0]))

score_d3ap_fear_loc = temp_pensioni.columns.get_loc('score_d3_fear')
temp_pensioni.insert(score_d3ap_fear_loc+1, "score_d3_sadness", np.zeros(temp_pensioni.shape[0]))

score_d3ap_sadness_loc = temp_pensioni.columns.get_loc('score_d3_sadness')
temp_pensioni.insert(score_d3ap_sadness_loc+1, "score_d3_joy", np.zeros(temp_pensioni.shape[0]))

score_d3ap_joy_loc = temp_pensioni.columns.get_loc('score_d3_joy')
temp_pensioni.insert(score_d3ap_joy_loc+1, "score_d3_positive_sent", np.zeros(temp_pensioni.shape[0]))

In [ ]:
# d3aperta_loc = temp_pensioni.columns.get_loc('d3aperta')
# temp_pensioni.insert(d3aperta_loc+1, "label_d3aperta_emotion", np.zeros(temp_pensioni.shape[0]))

# label_d3aperta_emotion_loc = temp_pensioni.columns.get_loc('label_d3aperta_emotion')
# temp_pensioni.insert(label_d3aperta_emotion_loc+1, "score_d3aperta_anger", np.zeros(temp_pensioni.shape[0]))

# score_d3aperta_anger_loc = temp_pensioni.columns.get_loc('score_d3aperta_anger')
# temp_pensioni.insert(score_d3aperta_anger_loc+1, "score_d3aperta_fear", np.zeros(temp_pensioni.shape[0]))

# score_d3aperta_fear_loc = temp_pensioni.columns.get_loc('score_d3aperta_fear')
# temp_pensioni.insert(score_d3aperta_fear_loc+1, "score_d3aperta_sadness", np.zeros(temp_pensioni.shape[0]))

# score_d3aperta_sadness_loc = temp_pensioni.columns.get_loc('score_d3aperta_sadness')
# temp_pensioni.insert(score_d3aperta_sadness_loc+1, "score_d3aperta_joy", np.zeros(temp_pensioni.shape[0]))

# score_d3aperta_joy_loc = temp_pensioni.columns.get_loc('score_d3aperta_joy')
# temp_pensioni.insert(score_d3aperta_joy_loc+1, "score_d3aperta_positive_sent", np.zeros(temp_pensioni.shape[0]))

In [ ]:
temp_pensioni.columns

Index(['IDU', 'STARTTIME', 'ENDTIME', 'TOTALDURATION', 'p_sesso', 'p_r_eta',
       'p_r_eta_6', 'generation', 'random', 'lega', 'p_comune_istat',
       'p_geo_cap', 'p_r_ampiezza6', 'r_capoluogo', 'a_istat_reg',
       'a_istat_prov', 'zona_5istat', 'p_occ_principale', 'p_prof',
       'p_autocol', 'p_voto_pol22', 'p_int_voto', 'p_scolarita', 'scol2', 'd0',
       'introaudio', 'data0', 'ora0', 'd1ap', 'label_d1_emotion',
       'score_d1_anger', 'score_d1_fear', 'score_d1_sadness', 'score_d1_joy',
       'score_d1_positive_sent', 'd1', 'd1aperta', 'd1recorder', 'd1rec',
       'data1', 'ora1', 'd2ap', 'label_d2_emotion', 'score_d2_anger',
       'score_d2_fear', 'score_d2_sadness', 'score_d2_joy',
       'score_d2_positive_sent', 'd2', 'd2aperta', 'd2recorder', 'd2rec',
       'data2', 'ora2', 'd3ap', 'label_d3_emotion', 'score_d3_anger',
       'score_d3_fear', 'score_d3_sadness', 'score_d3_joy',
       'score_d3_positive_sent', 'd3', 'd3aperta', 'd3recorder', 'd3rec',
       'data

##Transform to list

In [ ]:
temp_pensioni["label_d1_emotion"] = temp_pensioni["label_d1_emotion"].apply(lambda x: [x])
temp_pensioni["label_d2_emotion"] = temp_pensioni["label_d2_emotion"].apply(lambda x: [x])
temp_pensioni["label_d3_emotion"] = temp_pensioni["label_d3_emotion"].apply(lambda x: [x])

temp_pensioni["score_d1_anger"] = temp_pensioni["score_d1_anger"].apply(lambda x: [x])
temp_pensioni["score_d2_anger"] = temp_pensioni["score_d2_anger"].apply(lambda x: [x])
temp_pensioni["score_d3_anger"] = temp_pensioni["score_d3_anger"].apply(lambda x: [x])

temp_pensioni["score_d1_joy"] = temp_pensioni["score_d1_joy"].apply(lambda x: [x])
temp_pensioni["score_d2_joy"] = temp_pensioni["score_d2_joy"].apply(lambda x: [x])
temp_pensioni["score_d3_joy"] = temp_pensioni["score_d3_joy"].apply(lambda x: [x])

temp_pensioni["score_d1_sadness"] = temp_pensioni["score_d1_sadness"].apply(lambda x: [x])
temp_pensioni["score_d2_sadness"] = temp_pensioni["score_d2_sadness"].apply(lambda x: [x])
temp_pensioni["score_d3_sadness"] = temp_pensioni["score_d3_sadness"].apply(lambda x: [x])

temp_pensioni["score_d1_fear"] = temp_pensioni["score_d1_fear"].apply(lambda x: [x])
temp_pensioni["score_d2_fear"] = temp_pensioni["score_d2_fear"].apply(lambda x: [x])
temp_pensioni["score_d3_fear"] = temp_pensioni["score_d3_fear"].apply(lambda x: [x])

temp_pensioni["score_d1_positive_sent"] = temp_pensioni["score_d1_positive_sent"].apply(lambda x: [x])
temp_pensioni["score_d2_positive_sent"] = temp_pensioni["score_d2_positive_sent"].apply(lambda x: [x])
temp_pensioni["score_d3_positive_sent"] = temp_pensioni["score_d3_positive_sent"].apply(lambda x: [x])

#Classification

In [ ]:
%%capture
em_classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-emotion',top_k=4)
sent_classifier = pipeline("text-classification",model='MilaNLProc/feel-it-italian-sentiment',top_k=2)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
pensioni_df.columns

Index(['IDU', 'STARTTIME', 'ENDTIME', 'TOTALDURATION', 'p_sesso', 'p_r_eta',
       'p_r_eta_6', 'generation', 'random', 'lega', 'p_comune_istat',
       'p_geo_cap', 'p_r_ampiezza6', 'r_capoluogo', 'a_istat_reg',
       'a_istat_prov', 'zona_5istat', 'p_occ_principale', 'p_prof',
       'p_autocol', 'p_voto_pol22', 'p_int_voto', 'p_scolarita', 'scol2', 'd0',
       'introaudio', 'data0', 'ora0', 'd1ap', 'd1', 'd1aperta', 'd1recorder',
       'd1rec', 'data1', 'ora1', 'd2ap', 'd2', 'd2aperta', 'd2recorder',
       'd2rec', 'data2', 'ora2', 'd3ap', 'd3', 'd3aperta', 'd3recorder',
       'd3rec', 'data3', 'ora3', 'etapensione', 'annicontributi', 'platform'],
      dtype='object')

In [ ]:
audio_df.columns

Index(['IDU', 'd1rec_transcript', 'd1rec_significative',
       'd1rec_significative_num', 'd2rec_transcript', 'd2rec_significative',
       'd2rec_significative_num', 'd3rec_transcript', 'd3rec_significative',
       'd3rec_significative_num'],
      dtype='object')

#lable_d1ap_emotion, lable_d2ap_emotion, lable_d3ap_emotion

In [ ]:
for index, row in temp_pensioni.iterrows():
  if not pd.isnull(pensioni_df.at[index, "d1ap"]):
    temp_pensioni.at[index, "label_d1_emotion"] = em_classifier(pensioni_df.at[index, "d1ap"])[0][0]["label"] if isinstance(pensioni_df.at[index, "d1ap"], str) else float("nan")
  else:
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d1"] == "Preferisco scrivere":
        temp_pensioni.at[index, "label_d1_emotion"] = em_classifier(pensioni_df.at[index, "d1aperta"])[0][0]["label"] if isinstance(pensioni_df.at[index, "d1aperta"], str) else float("nan")
      elif pensioni_df.at[index, "d1"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d1rec_transcript'].values[0]
          temp_pensioni.at[index, "label_d1_emotion"] = em_classifier(transcript)[0][0]["label"] if isinstance(transcript, str) else float("nan")
        elif row_with_idu.empty:
          print('No row with IDU =', idu)
          temp_pensioni.at[index, "label_d1_emotion"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d1rec_transcript'].values[0]
        temp_pensioni.at[index, "label_d1_emotion"] = em_classifier(transcript)[0][0]["label"] if isinstance(transcript, str) else float("nan")
      elif row_with_idu.empty:
        print('No row with IDU =', idu)
        temp_pensioni.at[index, "label_d1_emotion"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

In [ ]:
for index, row in temp_pensioni.iterrows():
  if not pd.isnull(pensioni_df.at[index, "d2ap"]):
    temp_pensioni.at[index, "label_d2_emotion"] = em_classifier(pensioni_df.at[index, "d2ap"])[0][0]["label"] if isinstance(pensioni_df.at[index, "d2ap"], str) else float("nan")
  else:
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d2"] == "Preferisco scrivere":
        temp_pensioni.at[index, "label_d2_emotion"] = em_classifier(pensioni_df.at[index, "d2aperta"])[0][0]["label"] if isinstance(pensioni_df.at[index, "d2aperta"], str) else float("nan")
      elif pensioni_df.at[index, "d2"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d2rec_transcript'].values[0]
          temp_pensioni.at[index, "label_d2_emotion"] = em_classifier(transcript)[0][0]["label"] if isinstance(transcript, str) else float("nan")
        elif row_with_idu.empty:
          print('No row with IDU =', idu)
          temp_pensioni.at[index, "label_d2_emotion"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d2rec_transcript'].values[0]
        temp_pensioni.at[index, "label_d2_emotion"] = em_classifier(transcript)[0][0]["label"] if isinstance(transcript, str) else float("nan")
      elif row_with_idu.empty:
        print('No row with IDU =', idu)
        temp_pensioni.at[index, "label_d2_emotion"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

In [ ]:
for index, row in temp_pensioni.iterrows():
  if not pd.isnull(pensioni_df.at[index, "d3ap"]):
    temp_pensioni.at[index, "label_d3_emotion"] = em_classifier(pensioni_df.at[index, "d3ap"])[0][0]["label"] if isinstance(pensioni_df.at[index, "d3ap"], str) else float("nan")
  else:
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d3"] == "Preferisco scrivere":
        temp_pensioni.at[index, "label_d3_emotion"] = em_classifier(pensioni_df.at[index, "d3aperta"])[0][0]["label"] if isinstance(pensioni_df.at[index, "d3aperta"], str) else float("nan")
      elif pensioni_df.at[index, "d3"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d3rec_transcript'].values[0]
          temp_pensioni.at[index, "label_d3_emotion"] = em_classifier(transcript)[0][0]["label"] if isinstance(transcript, str) else float("nan")
        elif row_with_idu.empty:
          print('No row with IDU =', idu)
          temp_pensioni.at[index, "label_d3_emotion"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d3rec_transcript'].values[0]
        temp_pensioni.at[index, "label_d3_emotion"] = em_classifier(transcript)[0][0]["label"] if isinstance(transcript, str) else float("nan")
      elif row_with_idu.empty:
        print('No row with IDU =', idu)
        temp_pensioni.at[index, "label_d3_emotion"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

In [ ]:
temp_pensioni["label_d1_emotion"].to_list()[-5]

In [ ]:
temp_pensioni["label_d2_emotion"].to_list()

In [ ]:
temp_pensioni["label_d3_emotion"].to_list()

# Emotion Scores

In [ ]:
for index, row in temp_pensioni.iterrows():
  if isinstance(pensioni_df.at[index, "d1ap"], str):
    for d in em_classifier(pensioni_df.at[index, "d1ap"])[0]:
      if d["label"] == "anger":
        anger_score = d["score"]
      if d["label"] == "joy":
        joy_score = d["score"]
      if d["label"] == "fear":
        fear_score = d["score"]
      elif d["label"] == "sadness":
        sadness_score = d["score"]
    temp_pensioni.at[index, "score_d1_anger"] = anger_score
    temp_pensioni.at[index, "score_d1_joy"] = joy_score
    temp_pensioni.at[index, "score_d1_fear"] = fear_score
    temp_pensioni.at[index, "score_d1_sadness"] = sadness_score
  elif pd.isnull(pensioni_df.at[index, "d1ap"]):
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d1"] == "Preferisco scrivere":
        if isinstance(pensioni_df.at[index, "d1aperta"], str):
          for d in em_classifier(pensioni_df.at[index, "d1aperta"])[0]:
            if d["label"] == "anger":
              anger_score = d["score"]
            if d["label"] == "joy":
              joy_score = d["score"]
            if d["label"] == "fear":
              fear_score = d["score"]
            elif d["label"] == "sadness":
              sadness_score = d["score"]
          temp_pensioni.at[index, "score_d1_anger"] = anger_score
          temp_pensioni.at[index, "score_d1_joy"] = joy_score
          temp_pensioni.at[index, "score_d1_fear"] = fear_score
          temp_pensioni.at[index, "score_d1_sadness"] = sadness_score
      elif pensioni_df.at[index, "d1"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d1rec_transcript'].values[0]
          if isinstance(transcript, str):
            for d in em_classifier(transcript)[0]:
              if d["label"] == "anger":
                anger_score = d["score"]
              if d["label"] == "joy":
                joy_score = d["score"]
              if d["label"] == "fear":
                fear_score = d["score"]
              elif d["label"] == "sadness":
                sadness_score = d["score"]
            temp_pensioni.at[index, "score_d1_anger"] = anger_score
            temp_pensioni.at[index, "score_d1_joy"] = joy_score
            temp_pensioni.at[index, "score_d1_fear"] = fear_score
            temp_pensioni.at[index, "score_d1_sadness"] = sadness_score
          elif row_with_idu.empty or not isinstance(transcript, str):
            if row_with_idu.empty: print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d1_anger"] = float("nan")
            temp_pensioni.at[index, "score_d1_joy"] = float("nan")
            temp_pensioni.at[index, "score_d1_fear"] = float("nan")
            temp_pensioni.at[index, "score_d1_sadness"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d1rec_transcript'].values[0]
        if isinstance(transcript, str):
            for d in em_classifier(transcript)[0]:
              if d["label"] == "anger":
                anger_score = d["score"]
              if d["label"] == "joy":
                joy_score = d["score"]
              if d["label"] == "fear":
                fear_score = d["score"]
              elif d["label"] == "sadness":
                sadness_score = d["score"]
            temp_pensioni.at[index, "score_d1_anger"] = anger_score
            temp_pensioni.at[index, "score_d1_joy"] = joy_score
            temp_pensioni.at[index, "score_d1_fear"] = fear_score
            temp_pensioni.at[index, "score_d1_sadness"] = sadness_score
        elif row_with_idu.empty or not isinstance(transcript, str):
            if row_with_idu.empty: print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d1_anger"] = float("nan")
            temp_pensioni.at[index, "score_d1_joy"] = float("nan")
            temp_pensioni.at[index, "score_d1_fear"] = float("nan")
            temp_pensioni.at[index, "score_d1_sadness"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

In [ ]:
for index, row in temp_pensioni.iterrows():
  if isinstance(pensioni_df.at[index, "d2ap"], str):
    for d in em_classifier(pensioni_df.at[index, "d2ap"])[0]:
      if d["label"] == "anger":
        anger_score = d["score"]
      if d["label"] == "joy":
        joy_score = d["score"]
      if d["label"] == "fear":
        fear_score = d["score"]
      elif d["label"] == "sadness":
        sadness_score = d["score"]
    temp_pensioni.at[index, "score_d2_anger"] = anger_score
    temp_pensioni.at[index, "score_d2_joy"] = joy_score
    temp_pensioni.at[index, "score_d2_fear"] = fear_score
    temp_pensioni.at[index, "score_d2_sadness"] = sadness_score
  elif pd.isnull(pensioni_df.at[index, "d2ap"]):
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d2"] == "Preferisco scrivere":
        if isinstance(pensioni_df.at[index, "d2aperta"], str):
          for d in em_classifier(pensioni_df.at[index, "d2aperta"])[0]:
            if d["label"] == "anger":
              anger_score = d["score"]
            if d["label"] == "joy":
              joy_score = d["score"]
            if d["label"] == "fear":
              fear_score = d["score"]
            elif d["label"] == "sadness":
              sadness_score = d["score"]
          temp_pensioni.at[index, "score_d2_anger"] = anger_score
          temp_pensioni.at[index, "score_d2_joy"] = joy_score
          temp_pensioni.at[index, "score_d2_fear"] = fear_score
          temp_pensioni.at[index, "score_d2_sadness"] = sadness_score
      elif pensioni_df.at[index, "d2"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d2rec_transcript'].values[0]
          if isinstance(transcript, str):
            for d in em_classifier(transcript)[0]:
              if d["label"] == "anger":
                anger_score = d["score"]
              if d["label"] == "joy":
                joy_score = d["score"]
              if d["label"] == "fear":
                fear_score = d["score"]
              elif d["label"] == "sadness":
                sadness_score = d["score"]
            temp_pensioni.at[index, "score_d2_anger"] = anger_score
            temp_pensioni.at[index, "score_d2_joy"] = joy_score
            temp_pensioni.at[index, "score_d2_fear"] = fear_score
            temp_pensioni.at[index, "score_d2_sadness"] = sadness_score
          elif row_with_idu.empty or not isinstance(transcript, str):
            if row_with_idu.empty: print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d2_anger"] = float("nan")
            temp_pensioni.at[index, "score_d2_joy"] = float("nan")
            temp_pensioni.at[index, "score_d2_fear"] = float("nan")
            temp_pensioni.at[index, "score_d2_sadness"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d2rec_transcript'].values[0]
        if isinstance(transcript, str):
            for d in em_classifier(transcript)[0]:
              if d["label"] == "anger":
                anger_score = d["score"]
              if d["label"] == "joy":
                joy_score = d["score"]
              if d["label"] == "fear":
                fear_score = d["score"]
              elif d["label"] == "sadness":
                sadness_score = d["score"]
            temp_pensioni.at[index, "score_d2_anger"] = anger_score
            temp_pensioni.at[index, "score_d2_joy"] = joy_score
            temp_pensioni.at[index, "score_d2_fear"] = fear_score
            temp_pensioni.at[index, "score_d2_sadness"] = sadness_score
        elif row_with_idu.empty or not isinstance(transcript, str):
            if row_with_idu.empty: print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d2_anger"] = float("nan")
            temp_pensioni.at[index, "score_d2_joy"] = float("nan")
            temp_pensioni.at[index, "score_d2_fear"] = float("nan")
            temp_pensioni.at[index, "score_d2_sadness"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

In [ ]:
for index, row in temp_pensioni.iterrows():
  if isinstance(pensioni_df.at[index, "d3ap"], str):
    for d in em_classifier(pensioni_df.at[index, "d3ap"])[0]:
      if d["label"] == "anger":
        anger_score = d["score"]
      if d["label"] == "joy":
        joy_score = d["score"]
      if d["label"] == "fear":
        fear_score = d["score"]
      elif d["label"] == "sadness":
        sadness_score = d["score"]
    temp_pensioni.at[index, "score_d3_anger"] = anger_score
    temp_pensioni.at[index, "score_d3_joy"] = joy_score
    temp_pensioni.at[index, "score_d3_fear"] = fear_score
    temp_pensioni.at[index, "score_d3_sadness"] = sadness_score
  elif pd.isnull(pensioni_df.at[index, "d3ap"]):
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d3"] == "Preferisco scrivere":
        if isinstance(pensioni_df.at[index, "d3aperta"], str):
          for d in em_classifier(pensioni_df.at[index, "d3aperta"])[0]:
            if d["label"] == "anger":
              anger_score = d["score"]
            if d["label"] == "joy":
              joy_score = d["score"]
            if d["label"] == "fear":
              fear_score = d["score"]
            elif d["label"] == "sadness":
              sadness_score = d["score"]
          temp_pensioni.at[index, "score_d3_anger"] = anger_score
          temp_pensioni.at[index, "score_d3_joy"] = joy_score
          temp_pensioni.at[index, "score_d3_fear"] = fear_score
          temp_pensioni.at[index, "score_d3_sadness"] = sadness_score
      elif pensioni_df.at[index, "d3"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d3rec_transcript'].values[0]
          if isinstance(transcript, str):
            for d in em_classifier(transcript)[0]:
              if d["label"] == "anger":
                anger_score = d["score"]
              if d["label"] == "joy":
                joy_score = d["score"]
              if d["label"] == "fear":
                fear_score = d["score"]
              elif d["label"] == "sadness":
                sadness_score = d["score"]
            temp_pensioni.at[index, "score_d3_anger"] = anger_score
            temp_pensioni.at[index, "score_d3_joy"] = joy_score
            temp_pensioni.at[index, "score_d3_fear"] = fear_score
            temp_pensioni.at[index, "score_d3_sadness"] = sadness_score
          elif row_with_idu.empty or not isinstance(transcript, str):
            if row_with_idu.empty: print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d3_anger"] = float("nan")
            temp_pensioni.at[index, "score_d3_joy"] = float("nan")
            temp_pensioni.at[index, "score_d3_fear"] = float("nan")
            temp_pensioni.at[index, "score_d3_sadness"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d3rec_transcript'].values[0]
        if isinstance(transcript, str):
            for d in em_classifier(transcript)[0]:
              if d["label"] == "anger":
                anger_score = d["score"]
              if d["label"] == "joy":
                joy_score = d["score"]
              if d["label"] == "fear":
                fear_score = d["score"]
              elif d["label"] == "sadness":
                sadness_score = d["score"]
            temp_pensioni.at[index, "score_d3_anger"] = anger_score
            temp_pensioni.at[index, "score_d3_joy"] = joy_score
            temp_pensioni.at[index, "score_d3_fear"] = fear_score
            temp_pensioni.at[index, "score_d3_sadness"] = sadness_score
        elif row_with_idu.empty or not isinstance(transcript, str):
            if row_with_idu.empty: print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d3_anger"] = float("nan")
            temp_pensioni.at[index, "score_d3_joy"] = float("nan")
            temp_pensioni.at[index, "score_d3_fear"] = float("nan")
            temp_pensioni.at[index, "score_d3_sadness"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

# score_d1,2,3_positive_sent

In [ ]:
sent_classifier("ciao")

[[{'label': 'negative', 'score': 0.9992262125015259},
  {'label': 'positive', 'score': 0.0007737715495750308}]]

In [ ]:
for index, row in temp_pensioni.iterrows():
  if isinstance(pensioni_df.at[index, "d1ap"], str):
    for d in sent_classifier(pensioni_df.at[index, "d1ap"])[0]:
      if d["label"] == "positive":
        positive_score = d["score"]
    temp_pensioni.at[index, "score_d1_positive_sent"] = positive_score
  elif pd.isnull(pensioni_df.at[index, "d1ap"]):
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d1"] == "Preferisco scrivere":
        if isinstance(pensioni_df.at[index, "d1aperta"], str):
          for d in sent_classifier(pensioni_df.at[index, "d1aperta"])[0]:
            if d["label"] == "positive":
              positive_score = d["score"]
          temp_pensioni.at[index, "score_d1_positive_sent"] = positive_score
      elif pensioni_df.at[index, "d1"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d1rec_transcript'].values[0]
          if isinstance(transcript, str):
              for d in sent_classifier(transcript)[0]:
                if d["label"] == "positive":
                  positive_score = d["score"]
              temp_pensioni.at[index, "score_d1_positive_sent"] = positive_score
          elif not isinstance(transcript, str) or row_with_idu.empty:
            if row_with_idu.empty:
              print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d1_positive_sent"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d1rec_transcript'].values[0]
        if isinstance(transcript, str):
            for d in sent_classifier(transcript)[0]:
              if d["label"] == "positive":
                positive_score = d["score"]
            temp_pensioni.at[index, "score_d1_positive_sent"] = positive_score
        elif not isinstance(transcript, str) or row_with_idu.empty:
          if row_with_idu.empty:
            print(f'No row with IDU = {idu}')
          temp_pensioni.at[index, "score_d1_positive_sent"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

In [ ]:
for index, row in temp_pensioni.iterrows():
  if isinstance(pensioni_df.at[index, "d2ap"], str):
    for d in sent_classifier(pensioni_df.at[index, "d2ap"])[0]:
      if d["label"] == "positive":
        positive_score = d["score"]
    temp_pensioni.at[index, "score_d2_positive_sent"] = positive_score
  elif pd.isnull(pensioni_df.at[index, "d2ap"]):
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d2"] == "Preferisco scrivere":
        if isinstance(pensioni_df.at[index, "d2aperta"], str):
          for d in sent_classifier(pensioni_df.at[index, "d2aperta"])[0]:
            if d["label"] == "positive":
              positive_score = d["score"]
          temp_pensioni.at[index, "score_d2_positive_sent"] = positive_score
      elif pensioni_df.at[index, "d2"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d2rec_transcript'].values[0]
          if isinstance(transcript, str):
              for d in sent_classifier(transcript)[0]:
                if d["label"] == "positive":
                  positive_score = d["score"]
              temp_pensioni.at[index, "score_d2_positive_sent"] = positive_score
          elif not isinstance(transcript, str) or row_with_idu.empty:
            if row_with_idu.empty:
              print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d2_positive_sent"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d2rec_transcript'].values[0]
        if isinstance(transcript, str):
            for d in sent_classifier(transcript)[0]:
              if d["label"] == "positive":
                positive_score = d["score"]
            temp_pensioni.at[index, "score_d2_positive_sent"] = positive_score
        elif not isinstance(transcript, str) or row_with_idu.empty:
          if row_with_idu.empty:
            print(f'No row with IDU = {idu}')
          temp_pensioni.at[index, "score_d2_positive_sent"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

Took care of 0 lines out of 1235
Took care of 50 lines out of 1235
Took care of 100 lines out of 1235
Took care of 150 lines out of 1235
Took care of 200 lines out of 1235
Took care of 250 lines out of 1235
Took care of 300 lines out of 1235
Took care of 350 lines out of 1235
Took care of 400 lines out of 1235
Took care of 450 lines out of 1235
Took care of 500 lines out of 1235
Took care of 550 lines out of 1235
Took care of 600 lines out of 1235
Took care of 650 lines out of 1235
Took care of 700 lines out of 1235
Took care of 750 lines out of 1235
Took care of 800 lines out of 1235
Took care of 850 lines out of 1235
Took care of 900 lines out of 1235
Took care of 950 lines out of 1235
Took care of 1000 lines out of 1235
Took care of 1050 lines out of 1235
Took care of 1100 lines out of 1235
Took care of 1150 lines out of 1235
Took care of 1200 lines out of 1235


In [ ]:
for index, row in temp_pensioni.iterrows():
  if isinstance(pensioni_df.at[index, "d3ap"], str):
    for d in sent_classifier(pensioni_df.at[index, "d3ap"])[0]:
      if d["label"] == "positive":
        positive_score = d["score"]
    temp_pensioni.at[index, "score_d3_positive_sent"] = positive_score
  elif pd.isnull(pensioni_df.at[index, "d3ap"]):
    if pensioni_df.at[index, "random"] == "Domande con scelta del panelista tra aperte e registrazione audio":
      if pensioni_df.at[index, "d3"] == "Preferisco scrivere":
        if isinstance(pensioni_df.at[index, "d3aperta"], str):
          for d in sent_classifier(pensioni_df.at[index, "d3aperta"])[0]:
            if d["label"] == "positive":
              positive_score = d["score"]
          temp_pensioni.at[index, "score_d3_positive_sent"] = positive_score
      elif pensioni_df.at[index, "d3"] == "Desidero rispondere registrando un audio":
        idu = pensioni_df.at[index, "IDU"]
        row_with_idu = choseaudio_df[choseaudio_df['IDU'] == idu]
        if not row_with_idu.empty:
          transcript = row_with_idu['d3rec_transcript'].values[0]
          if isinstance(transcript, str):
              for d in sent_classifier(transcript)[0]:
                if d["label"] == "positive":
                  positive_score = d["score"]
              temp_pensioni.at[index, "score_d3_positive_sent"] = positive_score
          elif not isinstance(transcript, str) or row_with_idu.empty:
            if row_with_idu.empty:
              print(f'No row with IDU = {idu}')
            temp_pensioni.at[index, "score_d3_positive_sent"] = float("nan")
    elif pensioni_df.at[index, "random"] == "Domande con registrazione audio":
      idu = pensioni_df.at[index, "IDU"]
      row_with_idu = audio_df[audio_df['IDU'] == idu]
      if not row_with_idu.empty:
        transcript = row_with_idu['d3rec_transcript'].values[0]
        if isinstance(transcript, str):
            for d in sent_classifier(transcript)[0]:
              if d["label"] == "positive":
                positive_score = d["score"]
            temp_pensioni.at[index, "score_d3_positive_sent"] = positive_score
        elif not isinstance(transcript, str) or row_with_idu.empty:
          if row_with_idu.empty:
            print(f'No row with IDU = {idu}')
          temp_pensioni.at[index, "score_d3_positive_sent"] = float("nan")
  if index % 50 == 0:
    print(f"Took care of {index} lines out of {temp_pensioni.shape[0]}")

Took care of 0 lines out of 1235
Took care of 50 lines out of 1235
Took care of 100 lines out of 1235
Took care of 150 lines out of 1235
Took care of 200 lines out of 1235
Took care of 250 lines out of 1235
Took care of 300 lines out of 1235
Took care of 350 lines out of 1235
Took care of 400 lines out of 1235
Took care of 450 lines out of 1235
Took care of 500 lines out of 1235
Took care of 550 lines out of 1235
Took care of 600 lines out of 1235
Took care of 650 lines out of 1235
Took care of 700 lines out of 1235
Took care of 750 lines out of 1235
Took care of 800 lines out of 1235
Took care of 850 lines out of 1235
Took care of 900 lines out of 1235
Took care of 950 lines out of 1235
Took care of 1000 lines out of 1235
Took care of 1050 lines out of 1235
Took care of 1100 lines out of 1235
Took care of 1150 lines out of 1235
Took care of 1200 lines out of 1235


# Sanity checks

In [ ]:
temp_pensioni.columns

Index(['IDU', 'STARTTIME', 'ENDTIME', 'TOTALDURATION', 'p_sesso', 'p_r_eta',
       'p_r_eta_6', 'generation', 'random', 'lega', 'p_comune_istat',
       'p_geo_cap', 'p_r_ampiezza6', 'r_capoluogo', 'a_istat_reg',
       'a_istat_prov', 'zona_5istat', 'p_occ_principale', 'p_prof',
       'p_autocol', 'p_voto_pol22', 'p_int_voto', 'p_scolarita', 'scol2', 'd0',
       'introaudio', 'data0', 'ora0', 'd1ap', 'label_d1_emotion',
       'score_d1_anger', 'score_d1_fear', 'score_d1_sadness', 'score_d1_joy',
       'score_d1_positive_sent', 'd1', 'd1aperta', 'd1recorder', 'd1rec',
       'data1', 'ora1', 'd2ap', 'label_d2_emotion', 'score_d2_anger',
       'score_d2_fear', 'score_d2_sadness', 'score_d2_joy',
       'score_d2_positive_sent', 'd2', 'd2aperta', 'd2recorder', 'd2rec',
       'data2', 'ora2', 'd3ap', 'label_d3_emotion', 'score_d3_anger',
       'score_d3_fear', 'score_d3_sadness', 'score_d3_joy',
       'score_d3_positive_sent', 'd3', 'd3aperta', 'd3recorder', 'd3rec',
       'data

In [ ]:
temp_pensioni[["IDU", "d1ap", "d1aperta", "d1recorder", "d1rec", "label_d1_emotion", "score_d1_anger", "score_d1_fear", "score_d1_sadness", "score_d1_joy", "score_d1_positive_sent"]].head()

In [ ]:
temp_pensioni[["IDU", "d2ap", "d2aperta", "d2recorder", "d2rec", "label_d2_emotion", "score_d2_anger", "score_d2_fear", "score_d2_sadness", "score_d2_joy", "score_d2_positive_sent"]].head()

,IDU,d2ap,d2aperta,d2recorder,d2rec,label_d2_emotion,score_d2_anger,score_d2_fear,score_d2_sadness,score_d2_joy,score_d2_positive_sent
0,NaN,"{INCLUDEBOTH question_id=""lega"" qstxpath=""stri...",NaN,Cliccando sul pulsante partirà la registrazion...,"{INCLUDEBOTH question_id=""lega"" qstxpath=""stri...",sadness,0.000726,0.071265,0.541276,0.386734,0.998733
1,775.0,NaN,NaN,NaN,d2_775.mp3,anger,0.997759,0.000534,0.001657,0.00005,0.000222
2,783.0,NaN,Non conosco a fondo la questione per cui non p...,NaN,NaN,sadness,0.002418,0.002952,0.993766,0.000864,0.001494
3,3019.0,NaN,Qualsiasi riforma deve esser complessiva ripor...,NaN,NaN,anger,0.975465,0.018257,0.006094,0.000184,0.000525
4,8589.0,NaN,Non è una buona soluzione,NaN,NaN,sadness,0.002257,0.32045,0.551603,0.125689,0.999496


In [ ]:
em_classifier("Qualsiasi riforma deve esser complessiva riportando ad equità tutte le categorie di pensionati; non solo quelli del presente e del futuro; ma coinvolgendo anche i pensionati precedenti.")

[[{'label': 'anger', 'score': 0.9754653573036194},
  {'label': 'fear', 'score': 0.018257256597280502},
  {'label': 'sadness', 'score': 0.006093719508498907},
  {'label': 'joy', 'score': 0.00018364850257057697}]]

In [ ]:
sent_classifier("Qualsiasi riforma deve esser complessiva riportando ad equità tutte le categorie di pensionati; non solo quelli del presente e del futuro; ma coinvolgendo anche i pensionati precedenti.")

[[{'label': 'negative', 'score': 0.9994749426841736},
  {'label': 'positive', 'score': 0.000525072100572288}]]

In [ ]:
temp_pensioni[["IDU", "d3ap", "d3aperta", "d3recorder", "d3rec", "label_d3_emotion", "score_d3_anger", "score_d3_fear", "score_d3_sadness", "score_d3_joy", "score_d3_positive_sent"]].head()

,IDU,d3ap,d3aperta,d3recorder,d3rec,label_d3_emotion,score_d3_anger,score_d3_fear,score_d3_sadness,score_d3_joy,score_d3_positive_sent
0,NaN,"Secondo lei, quale dovrebbe essere l’età raggi...",NaN,Cliccando sul pulsante partirà la registrazion...,"Secondo lei, quale dovrebbe essere l’età raggi...",fear,0.008791,0.899671,0.087254,0.004284,0.000943
1,775.0,NaN,NaN,NaN,d3_775.mp3,anger,0.793623,0.037074,0.168956,0.000348,0.000365
2,783.0,NaN,Penso che 67 68 anni siano l'età giusta indipe...,NaN,NaN,joy,0.000662,0.050363,0.005556,0.943419,0.998638
3,3019.0,NaN,40 anni di contributi: in tal modo chi è entra...,NaN,NaN,anger,0.954333,0.011119,0.034411,0.000138,0.000455
4,8589.0,NaN,Quarant'anni di contributi effettivi Età mass...,NaN,NaN,fear,0.027803,0.64221,0.326663,0.003325,0.501758


In [ ]:
sent_classifier("Penso che 67 68 anni siano l'età giusta indipendentemente dal numero di anni di contributi")

[[{'label': 'positive', 'score': 0.9986383318901062},
  {'label': 'negative', 'score': 0.0013616009382531047}]]

In [ ]:
em_classifier("Penso che 67 68 anni siano l'età giusta indipendentemente dal numero di anni di contributi")

[[{'label': 'joy', 'score': 0.9434187412261963},
  {'label': 'fear', 'score': 0.05036298558115959},
  {'label': 'sadness', 'score': 0.005555696319788694},
  {'label': 'anger', 'score': 0.0006624521338380873}]]

In [ ]:
temp_pensioni.columns

Index(['IDU', 'STARTTIME', 'ENDTIME', 'TOTALDURATION', 'p_sesso', 'p_r_eta',
       'p_r_eta_6', 'generation', 'random', 'lega', 'p_comune_istat',
       'p_geo_cap', 'p_r_ampiezza6', 'r_capoluogo', 'a_istat_reg',
       'a_istat_prov', 'zona_5istat', 'p_occ_principale', 'p_prof',
       'p_autocol', 'p_voto_pol22', 'p_int_voto', 'p_scolarita', 'scol2', 'd0',
       'introaudio', 'data0', 'ora0', 'd1ap', 'label_d1_emotion',
       'score_d1_anger', 'score_d1_fear', 'score_d1_sadness', 'score_d1_joy',
       'score_d1_positive_sent', 'd1', 'd1aperta', 'd1recorder', 'd1rec',
       'data1', 'ora1', 'd2ap', 'label_d2_emotion', 'score_d2_anger',
       'score_d2_fear', 'score_d2_sadness', 'score_d2_joy',
       'score_d2_positive_sent', 'd2', 'd2aperta', 'd2recorder', 'd2rec',
       'data2', 'ora2', 'd3ap', 'label_d3_emotion', 'score_d3_anger',
       'score_d3_fear', 'score_d3_sadness', 'score_d3_joy',
       'score_d3_positive_sent', 'd3', 'd3aperta', 'd3recorder', 'd3rec',
       'data

In [ ]:
temp_pensioni.to_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/FinalSentimentAnalysis_Pensioni.xlsx", index=False)

In [ ]:
sent_classifier("65 ")

[[{'label': 'positive', 'score': 0.999651312828064},
  {'label': 'negative', 'score': 0.00034874037373811007}]]

In [ ]:
sent_classifier("55")

[[{'label': 'negative', 'score': 0.9996616840362549},
  {'label': 'positive', 'score': 0.00033829614403657615}]]

In [ ]:
temp_pensioni_copy = temp_pensioni.copy(deep=True)

In [ ]:
temp_pensioni_copy.columns

In [ ]:
columns_to_add = ["d1rec_transcript", "d1rec_significative", "d1rec_significative_num",
                  "d2rec_transcript", "d2rec_significative", "d2rec_significative_num",
                  "d3rec_transcript", "d3rec_significative", "d3rec_significative_num"]

for column in columns_to_add:
    if 'significative' in column:
        temp_pensioni_copy[column] = [list()] * len(temp_pensioni_copy)
    else:
        temp_pensioni_copy[column] = ""

merged_df = temp_pensioni_copy.merge(stacked_df_audio, on='IDU', how='left', suffixes=('', '_y'))

for column in columns_to_add:
    merged_df[column] = merged_df[column + '_y'].combine_first(merged_df[column])

merged_df.drop([col + '_y' for col in columns_to_add], axis=1, inplace=True)

temp_pensioni_copy = merged_df

In [ ]:
columns_to_add = ['d1aperta_significative', 'd1aperta_significative_num',
                  'd2aperta_significative', 'd2aperta_significative_num',
                  'd3aperta_significative', 'd3aperta_significative_num']

for column in columns_to_add:
    if 'significative' in column:
        temp_pensioni_copy[column] = [list()] * len(temp_pensioni_copy)
    else:
        temp_pensioni_copy[column] = ""

merged_df_2 = temp_pensioni_copy.merge(stacked_df_open, on='IDU', how='left', suffixes=('', '_y'))

for column in columns_to_add:
    merged_df_2[column] = merged_df_2[column + '_y'].combine_first(merged_df_2[column])

merged_df_2.drop([col + '_y' for col in columns_to_add], axis=1, inplace=True)

temp_pensioni_copy = merged_df_2

In [ ]:
temp_pensioni_copy.columns

Index(['IDU', 'STARTTIME', 'ENDTIME', 'TOTALDURATION', 'p_sesso', 'p_r_eta',
       'p_r_eta_6', 'generation', 'random', 'lega', 'p_comune_istat',
       'p_geo_cap', 'p_r_ampiezza6', 'r_capoluogo', 'a_istat_reg',
       'a_istat_prov', 'zona_5istat', 'p_occ_principale', 'p_prof',
       'p_autocol', 'p_voto_pol22', 'p_int_voto', 'p_scolarita', 'scol2', 'd0',
       'introaudio', 'data0', 'ora0', 'd1ap', 'label_d1_emotion',
       'score_d1_anger', 'score_d1_fear', 'score_d1_sadness', 'score_d1_joy',
       'score_d1_positive_sent', 'd1', 'd1aperta', 'd1recorder', 'd1rec',
       'data1', 'ora1', 'd2ap', 'label_d2_emotion', 'score_d2_anger',
       'score_d2_fear', 'score_d2_sadness', 'score_d2_joy',
       'score_d2_positive_sent', 'd2', 'd2aperta', 'd2recorder', 'd2rec',
       'data2', 'ora2', 'd3ap', 'label_d3_emotion', 'score_d3_anger',
       'score_d3_fear', 'score_d3_sadness', 'score_d3_joy',
       'score_d3_positive_sent', 'd3', 'd3aperta', 'd3recorder', 'd3rec',
       'data

Fix the following columns (esthetic purpose)

In [ ]:
columns_to_modify = ["d1rec_significative_num", "d2rec_significative_num", "d3rec_significative_num",
                     "d1aperta_significative_num", "d2aperta_significative_num", "d3aperta_significative_num"]

temp_pensioni_copy[columns_to_modify] = temp_pensioni_copy[columns_to_modify].applymap(modify)

In [ ]:
columns_to_modify2 = ["d1aperta_significative", "d2aperta_significative", "d3aperta_significative",
                     "d1rec_significative", "d2rec_significative", "d3rec_significative"]

temp_pensioni_copy[columns_to_modify2] = temp_pensioni_copy[columns_to_modify2].applymap(modify2)

In [ ]:
stacked_df_audio = stack_dataframes(temp_choseaudio_df, temp_audio_df)
stacked_df_open = stack_dataframes(temp_choseopen_df, temp_open_df)

#Exporting file

In [ ]:
temp_pensioni_copy.to_excel("/content/gdrive/MyDrive/Speech2Text/Pensioni/temp_FinalSentimentAnalysis_Pensioni_copy.xlsx", index=False)

#Functions

In [ ]:
def overlapping_elements(list1, list2):
    set1 = set(list1)
    set2 = set(list2)

    common_elements = set1 & set2

    if len(common_elements) > 0:
        print(f'The lists have common elements: {common_elements}')
        return True
    else:
        print('The lists do not have any common elements.')
        return False

In [ ]:
def stack_dataframes(df1, df2):
    if set(df1.columns) == set(df2.columns):
        df = pd.concat([df1, df2], ignore_index=True)
        return df
    else:
        return "DataFrames do not have the same columns"

In [ ]:
def modify(entry):
    if entry == []:
        return float("nan")
    elif isinstance(entry, float):
        return int(entry)
    else:
        return entry

In [ ]:
def modify2(entry):
    if entry == []:
        return float("nan")
    elif isinstance(entry, str) and entry.startswith('[') and entry.endswith(']'):
        try:
            return ast.literal_eval(entry)
        except:
            return entry
    else:
        return entry

In [ ]:
em_classifier("Secondo lei, quale dovrebbe essere l’età raggiunta la quale le persone devono poter andare in pensione? Oppure, in alternativa, il numero di anni di contributi, raggiunti i quali, le persone devono poter andare in pensione indipendentemente dall’età che hanno? Dica un numero e spieghi perché le sembra l’età giusta (o il numero di anni di contributi giusto).")

[[{'label': 'fear', 'score': 0.899671196937561},
  {'label': 'sadness', 'score': 0.08725354075431824},
  {'label': 'anger', 'score': 0.008791006170213223},
  {'label': 'joy', 'score': 0.00428432272747159}]]

In [ ]:
sent_classifier("Secondo lei, quale dovrebbe essere l’età raggiunta la quale le persone devono poter andare in pensione? Oppure, in alternativa, il numero di anni di contributi, raggiunti i quali, le persone devono poter andare in pensione indipendentemente dall’età che hanno? Dica un numero e spieghi perché le sembra l’età giusta (o il numero di anni di contributi giusto).")

[[{'label': 'negative', 'score': 0.9990572333335876},
  {'label': 'positive', 'score': 0.0009428266785107553}]]